In [1]:

import numpy as np
import os
import math
from scipy import signal,io,fftpack
import pandas as pd
import mne
import pickle
import neurokit2 as nk
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
import seaborn as sns
from pathlib import Path
import heartpy as hp
from scipy.stats import lognorm
import warnings


# Reading path for edf
root_edf_path = Path("/Users/deepatilwani/Documents/Autism/NewAnalysis/edfFiles_threeToSix")
        
edf_paths = list(root_edf_path.glob("*"))


In [2]:
# list file and directories
res = os.listdir(root_edf_path)
res = pd.DataFrame(res)
for i, j  in res.iterrows(): 
        res.loc[i, 0] =j[0].split('/')[-1].split('_')[0]
        
#finding all unique id's
uniqueid = pd.DataFrame(np.unique(res)).astype(int)
uniqueid.columns = ['Id']

#labels LL EL 
InfoLabels = pd.read_excel('ParticipantLabels_13sept2022.xlsx')
InfoLabels.dropna(inplace=True)

InfoLabels.columns = ['Id', 'Status']

uniqueid_labels = uniqueid.merge(InfoLabels, on ='Id')
#uniqueid_labels
#Total participants 115 (all EL, LL, PT and other)

In [4]:

#We have other groups than typically developed and elevated developed ... 
#Extracting the Typical (LL) and elevated  (EL ) labels and id's 

idsLLandEL = pd.DataFrame(uniqueid_labels[(uniqueid_labels['Status'] == 'LL') | (uniqueid_labels['Status'] == 'EL-SIB')])


idsLLandEL.to_csv('FinalIdswithLLandEL.csv', index= False)

#82  total participants after rejecting the PT and other. 

In [5]:
elll = pd.read_csv('FinalIdswithLLandEL.csv')
elll['Status'].value_counts()

#Files with no PIX and OIX 
#[1001, 1003, 1004, 1028, 1031, 1035, 1036, 1037, 1041, 1042, 1048, 1076]


#finding index 

index_ = elll.loc[elll['Id'].isin([1001, 1003, 1004, 1028, 1031, 1035, 1036, 1037, 1041, 1042, 1048, 1076])].index


ell = elll.drop(index_)
ell['Status'].value_counts()

LL        39
EL-SIB    31
Name: Status, dtype: int64

In [7]:
#Function to get the segments : OIX and PIX

#OIX : Object Interaction
#PIX : Parent Interaction

def get_segments(path_edf, seconds, **kwargs):
    
    
    """
    Input 
    
        path_edf : path to folder where all EDF files are stored
    
        seconds : We are dividing the segments into 30s,60s,90s,120s and 0(for full length) 
    
    
    Output 
        epoch_ecg : Seconds for the ECG segments multipe epochs set of [30,60,90,120,0]  
        condition : OIX or PIX
        subject   : Participant ID
        age       : In 3-6 months
        sfreq     : One of these 128,512 or 1024
        seconds   : One of these : 30,60,90,120, 0
    
    Author : Deepa Tilwani dtilwani@mailbox.sc.edu
    
    """
    

    # age is in months
    
    subject, age = path_edf.split('/')[-1].split('_')[0], path_edf.split('/')[-1].split('_')[1][:2]
    
    #print(subject, age)
    
    # get_log_times is a function which is extracting timestamps for the OIX and PIX experiments. 
    
    log_df = get_log_times(subject, age, **kwargs)
    
    #print(log_df)
    
    #Reading the EDF files
    edf_raw = mne.io.read_raw_edf(path_edf, preload=True)
    
    #Extracting the frequency 
    sfreq = edf_raw.info["sfreq"]
    
    #Intial filtering
    edf_raw = edf_raw.notch_filter(np.arange(60, sfreq/2.0, 60))
    edf_raw = edf_raw.filter(1, sfreq/4.0)

    
    #print(log_df.condition.values)
    
    if log_df is not None:
        #print("in if loop")
        #print(log_df.values)
        for i,j in log_df.iterrows(): 
                #Getting Stat/ stop of the experiment in seconds
                start_edf = int(j.start * 60)
                stop_edf =  int(j.end * 60)
                
                #print(start_edf,stop_edf)
                
                conditions = ['OIX', 'PIX', 'OIX2'] #OIX2 is also same as OIX, naming issues
                
                if j.condition in conditions:

                    if seconds == 0:
                        try:
                            print(j.condition)

                            segments = edf_raw.get_data("ECG0",tmin=start_edf,tmax=stop_edf).squeeze()
                            return  segments, j.condition ,subject,age ,  sfreq , seconds

                        except :
                            segments = edf_raw.get_data("ECG", tmin=start_edf,tmax=stop_edf).squeeze() 
                            return  segments, j.condition , subject,age,  sfreq, seconds

                    else : 
                    
                            print(seconds, start_edf, stop_edf)
                            epochs = mne.make_fixed_length_epochs(edf_raw.crop(tmin=start_edf,tmax=stop_edf),

                                                              duration=seconds, preload=False)

                            df_epoch = epochs.to_data_frame()
                            no_of_epoch = df_epoch.epoch.unique() 
                            #df_epoch have the epoch column which specify which segment the vaue is - 0,1,2 so on
                            array_for_epochs = []
                            for i in no_of_epoch:
                                epoch_ecg = df_epoch.loc[df_epoch['epoch'] == i].T.values
                                #Returning values in epoch_ecg ( indices have time, epoch number,segments(epochs))
                                return  epoch_ecg, j.condition , subject, age,  sfreq ,seconds 
                            
              

In [7]:
#old torsh timelogs - /Users/deepatilwani/Documents/Autism/NewAnalysis/all_infants_timelogs

# There was change in data saving pipeline in later of study. So, we have diffrent styles of time stamps files.


# Fuction to get the time logs of experiments OIX/PIX

def get_log_times(subject, age, path_timelog_format="/Users/deepatilwani/Documents/Autism/NewAnalysis/all_infants_timelogs/{subject}_{age}.csv",
                  datavyu_format="/Users/deepatilwani/Documents/Autism/Generated Files_{kind}_03092022_Datavyu_ALLOnly_AI/Stimuli/" +
                                 "{subject}_{age}_stimulus.csv"):

    # Look first for datavyu times
    rows = []
    for kind in ["OIX", "PIX", "OIX2"]:
        stim_path = Path(datavyu_format.format(kind=kind, subject=subject, age=age))
        if stim_path.exists():
            csv_file = pd.read_csv(stim_path).dropna()
            csv_file.columns = ["start", "end", "stimulus"]
            csv_file = csv_file[csv_file.stimulus != "END"]
            rows.append({"start": csv_file.start.min() / 60.0,
                         "end": csv_file.end.max() / 60.0,
                         "condition": kind})
      
    if len(rows):
        return pd.DataFrame(rows)

    # if datavyu times are not available, look for old time logs
    path_timelog = Path(path_timelog_format.format(subject=subject, age=age))
    if path_timelog.exists():
        csv_file = pd.read_csv(path_timelog).dropna()
        csv_file.columns = ["visit", "segment", "condition", "start", "end"]
        csv_file = csv_file[csv_file.end > csv_file.start]
        
        return csv_file

    # No segment logs available
    return None


In [ ]:
#Script to extract the data for all edf files 
data_extracted =[]
COUNT = 0

#defining the seconds for segmenting 
time_seconds  = [0,30,60,90,120]

errorfiles = []

for i in edf_paths:
    
    COUNT = COUNT+1
    subject = str(i.split('/')[-1].split('_')[0])
    age = i.split('/')[-1].split('_')[1][:2]
    name = subject+'_'+age
    
    for time in time_seconds:  
            if int(subject) in idsLLandEL["Id"].values: 
                    try:
                        segment, condition, edf_id, month, sfreq, seconds= get_segments(i,seconds=time ) 

                        data_extracted.append({ "Id" :edf_id,"month": month,"freq":sfreq,"segments":segment, "condition" : condition,
                                                     'segment_lenInSec': seconds}) 
                    except:

                        errorfiles.append(name)


In [12]:
#data_extracted is the variable name which have all edf's data 

# Converting into Dataframe

df = pd.DataFrame(data_extracted)
df = df.sort_values('Id').reset_index(drop=True)

#Saving it to pickel for later use
df.to_pickle("extracted_data_paper2023.pkl") 


#reading pickel
df_pickel = pd.read_pickle("extracted_data_paper2023.pkl")


In [ ]:
df_for_preprocess = []

df_pickel = df_pickel.sort_values(by = ['Id', 'segment_lenInSec'])

for i,j in df_pickel.iterrows():
    print(j)
  
    if not j['segment_lenInSec'] in [30,60,90,120]:
          print(j['segments'][3:])
        
          df_for_preprocess.append({ "Id" :j['Id'],"month": j['month'],"freq":j['freq'],
                               "segments":j['segments'][3:], "condition" : j['condition'],
                                      'segment_lenInSec': 0})
            
    else : 
        for x in j['segments'][3:]:

              df_for_preprocess.append({ "Id" :j['Id'],"month": j['month'],"freq":j['freq'],
                                   "segments":x, "condition" : j['condition'],
                                          'segment_lenInSec': j['segment_lenInSec']})
            
          
df_for_preprocess = pd.DataFrame(df_for_preprocess)
df_for_preprocess.to_pickle("df_for_preprocess2023.pkl") 

In [19]:
#To check the segments in each length 

df_for_preprocess = pd.read_pickle("df_for_preprocess2023.pkl")

grouped_df = df_for_preprocess.groupby('segment_lenInSec')

for key, item in grouped_df:
    if key==0:
        df_0 = grouped_df.get_group(0)
        print(len(df_0))
    if key==30:
        df_30 = grouped_df.get_group(30)
        print(len(df_30))
    if key==60:
        df_60 = grouped_df.get_group(60)
        print(len(df_60))
    if key==90:
        df_90 = grouped_df.get_group(90)
        print(len(df_90))
    if key==120:
        df_120 = grouped_df.get_group(120)
        print(len(df_120))

133
528
528
528
528


In [15]:
# Functions to Preprocess ECG files.

def get_outliers(vals, k=3):
    q1, q2, q3 = np.percentile(vals, [25, 50, 75])
    return (vals < q2 - k * (q3 - q1)) | (vals > q2 + k * (q3 - q1))


def process_ecg_data_segment(rec_ecg, sfreq, margin=250, debug=False, resample=None, **kwargs):
    if len(rec_ecg) == 0:
        return

    if resample is not None and resample != sfreq:
        rec_ecg = mne.filter.resample(rec_ecg, up=resample/sfreq)
        sfreq = resample

            
    try:
        # Using heartpy library to extract the Rpeaks
        wd, m = hp.process(rec_ecg, sfreq, **kwargs)
    except hp.exceptions.BadSignalWarning:
        return

    beats = []
    peaks, peaks_y = np.array([[peak, peak_y] for peak, peak_y in zip(wd["peaklist"], wd["ybeat"])
                               if peak not in wd["removed_beats"]]).T
    peaks = peaks.astype(int)

    p1s, p2s, p3s = peaks[:-2], peaks[1:-1], peaks[2:]
    for p1, p2, p3 in zip(p1s, p2s, p3s):
        x = np.arange(p1, p3 + 1)
        y = rec_ecg[p1:(p3 + 1)]
        f = interpolate.interp1d(x, y)

        xnew = np.concatenate((np.linspace(p1, p2, margin), np.linspace(p2, p3, margin)))
        ynew = f(xnew)  # use interpolation function returned by `interp1d`
        beats.append(ynew)

    beats = np.array(beats)

    if len(beats) == 0:
        return
    residuals = np.trapz((beats - beats.mean(0)) ** 2, axis=1)
    outliers = get_outliers(residuals, k=6)

    nb_samples = np.median((p3s - p1s)[~outliers])

    # Flagging as outliers P2 to close to the borders
    outliers |= ((p2s - nb_samples // 2).astype(int) < 0)
    outliers |= ((p2s + nb_samples // 2).astype(int) >= len(rec_ecg))

    additional_removed_beats = np.array(peaks)[np.concatenate([[True], outliers, [True]])]
    additional_removed_beats_y = np.array(peaks_y)[np.concatenate([[True], outliers, [True]])]

    clean_beats = beats[~outliers, :]

    raw_beats = np.array([rec_ecg[int(p2 - nb_samples // 2):int(p2 + nb_samples // 2)]
                          for p2 in p2s[~outliers]])

    raw_t = np.arange(-int(nb_samples // 2), int(nb_samples // 2)) / sfreq

    #if clean_beats.shape[0] < 20:
       # return

    wd_copy = wd.copy()

    wd_copy["removed_beats"] = np.array(np.concatenate([wd["removed_beats"], additional_removed_beats]))
    wd_copy["removed_beats_y"] = np.array(np.concatenate([wd["removed_beats_y"], additional_removed_beats_y]))
    #removed_beats -- > peaklist
    #removed_beats_y --> amlitude values of removed peaks
    clean_mean_beat = np.median(clean_beats, 0)

    signal = np.trapz(clean_mean_beat ** 2)
    noise = np.trapz((clean_beats - clean_mean_beat) ** 2, axis=1)

    if debug:
        plt.figure()
        hp.plotter(wd, m, figsize=(20, 4))
        plt.xlim(0, 30)

        plt.figure()
        plt.plot(residuals, ".")
        plt.plot(np.arange(len(residuals))[outliers], residuals[outliers], ".", color="r")

        plt.figure()
        hp.plotter(wd_copy, m, figsize=(20, 4))
        plt.xlim(0, 30)

        plt.figure()
        sns.heatmap(clean_beats)

        plt.figure()
        plt.plot(clean_beats.T, alpha=0.1, color='k')
        plt.plot(clean_mean_beat, color="r")
        
        


    return {"SNR": np.mean(10 * np.log10(signal / noise)),
    "mean_beat": clean_mean_beat,
    "nb_valid_beats": clean_beats.shape[0],
    "nb_invalid_beats": np.sum(outliers),
    # "file_parts": file_name.name.replace(".edf", "").split("_"),
    "wd": wd,
    "wd_copy": wd_copy, 
    "clean_beats": clean_beats, #beats
    "raw_beats": raw_beats,
    "raw_t": raw_t,
    "rel_p1": p2s[~outliers] - p1s[~outliers],
    "rel_p3": p3s[~outliers] - p2s[~outliers],
    "sfreq": sfreq}


In [ ]:
# fix this using the windowing for ECG (60s, 120s)

import scipy.interpolate as spi


invalid_segment_id = []
hrv_all = []

results_df = []
COUNT = 0
removedbeats = []
cleanbeats_list =[]
no_of_cleanbeats = []
perc_of_newbeats = []
for idx,row in df_for_preprocess.iterrows():
     #print(row)
    #preprocessing of the segments
  
    
    try :
        COUNT = COUNT+1
        #print(row['Id'],row['month'])
        
        pro = process_ecg_data_segment(row['segments'],sfreq=row['freq'],debug=False) #no resampling
        cleanbeats = [i for i in pro['wd_copy']['peaklist'] if i not in pro['wd_copy']['removed_beats']]
        cleanbeats_list.append(cleanbeats)
        removedbeats.append(pro['wd_copy']['removed_beats'])
        #print("Clean Beats :", len(cleanbeats))
        no_of_cleanbeats.append(len(cleanbeats))
        
        #Calculations for new beats interpolation percentage
        med_ibi = np.median(np.diff(cleanbeats))
        ibis = np.diff(cleanbeats)
        #print(ibis)
        nb_ibi = np.round(ibis/med_ibi).astype(int)
        #print(nb_ibi)

        x = np.cumsum(nb_ibi)[nb_ibi == 1]
        y = np.array(cleanbeats[1:])[nb_ibi == 1]
        f = spi.interp1d(x,y,kind = "linear",fill_value="extrapolate")
        xnew = np.arange(1, np.cumsum(nb_ibi)[-1]+1)
        new_peaks = np.round(f(xnew)) 

        #percentage of new beat
        perc = int((len(new_peaks)-len(cleanbeats))* 100/len(cleanbeats))
        #print("percentage of new beats :", perc)

        perc_of_newbeats.append(perc)

        if perc <=30:
            if len(cleanbeats) >= 30:
                #print("Increased percentage of peaks <= 30%", row['Id'],row['month'])            



                #print("NewPeaks",len(new_peaks))
                #print("Actual Cleaned Beats", len(cleanbeats)) 

                plt.plot(xnew, new_peaks, ".")
                plt.plot(x, y, ".")
                plt.show() 
                #Extracting HRV values on the basis of rpeak
                hrv_feat = nk.hrv(new_peaks, sampling_rate=sfreq, show=False)
                results_df.append({ "Id" :row['Id'],
                                "Month" :row['month'] ,
                                "MeanNN" : hrv_feat['HRV_MeanNN'][0],
                                "MedianNN": hrv_feat['HRV_MedianNN'][0], 
                                "pnn20": hrv_feat['HRV_pNN20'][0], 
                                "sd1sd2":hrv_feat['HRV_SD1SD2'][0],                                                                                  
                                "CvNN" : hrv_feat['HRV_CVNN'][0] , 
                                "HTI":hrv_feat['HRV_HTI'][0] , 
                                "CSI" :hrv_feat['HRV_CSI'] [0],
                                 "CVI":hrv_feat['HRV_CVI'][0],
                                  "MaxNN" : hrv_feat['HRV_MaxNN'][0],
                                  "MinNN": hrv_feat['HRV_MinNN'][0],
                                "condition" : row['condition'],
                                "segment_lenInSec" : int(row['segment_lenInSec'] )

                                 })

        
    except:
        invalid_segment_id.append(str(row['Id'])+'_'+row['month'])
          



In [23]:
sorted_result_df = pd.DataFrame(results_df).reset_index(drop=True)

#csv file-- for labels
ParticipantsInfo = pd.read_csv('ParticipantInfo_nodx.csv')

#labels LL EL 
InfoLabels = ParticipantsInfo[['Participant ID', 'Group']]

InfoLabels.dropna(inplace=True)

#changing label column names
InfoLabels.columns = [ 'Id', 'Labels']
sorted_result_df['Id'] = sorted_result_df['Id'].astype(int)

df_with_labels_segments = sorted_result_df.merge(InfoLabels, on ='Id')
df_with_labels_segments.to_pickle("df_with_labels_segments2023.pkl") 

/var/folders/71/lpprjh8x2x79bf69xl_jhvy40000gn/T/ipykernel_48881/756361255.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  InfoLabels.dropna(inplace=True)


In [24]:
pd.read_pickle("df_with_labels_segments2023.pkl")

,Id,Month,MeanNN,MedianNN,pnn20,sd1sd2,CvNN,HTI,CSI,CVI,MaxNN,MinNN,condition,segment_lenInSec,Labels
0,1002,6m,469.995959,457.031250,22.413793,0.550926,0.197166,3.411765,1.815126,5.038905,738.281250,248.046875,OIX,30,EL
1,1005,6m,445.312500,447.265625,8.474576,2.008370,0.098908,2.950000,0.497916,4.338518,625.000000,267.578125,OIX,30,LL
2,1007,6m,405.789022,404.296875,1.040312,0.208041,0.042334,4.660606,4.806756,3.270456,498.046875,355.468750,OIX,0,EL
3,1007,6m,408.264160,404.296875,1.562500,0.182949,0.053286,4.266667,5.466006,3.369886,498.046875,380.859375,OIX,30,EL
4,1007,6m,396.778682,394.531250,6.164384,0.812934,0.073757,5.840000,1.230112,4.124043,568.359375,250.000000,OIX,60,EL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,1133,4m,212.861032,212.890625,0.000000,0.297049,0.037210,2.750000,3.366453,2.740349,230.468750,199.218750,OIX,30,EL
465,1133,4m,207.938058,207.031250,0.000000,0.228353,0.043336,3.255814,4.379178,2.752455,230.468750,189.453125,OIX,60,EL
466,1133,4m,204.228154,203.125000,0.000000,0.202335,0.048234,3.176471,4.942306,2.778268,230.468750,183.593750,OIX,90,EL
467,1133,4m,201.618494,201.171875,0.682594,0.207661,0.053965,3.708861,4.815550,2.878500,230.468750,177.734375,OIX,120,EL


In [25]:
pd.read_pickle("df_with_labels_segments2023.pkl")
grouped_df = pd.read_pickle("df_with_labels_segments2023.pkl").groupby('segment_lenInSec')

for key, item in grouped_df:
    if key==0:
        df_0 = grouped_df.get_group(0)
        print(len(df_0))
    if key==30:
        df_30 = grouped_df.get_group(30)
        print(len(df_30))
    if key==60:
        df_60 = grouped_df.get_group(60)
        print(len(df_60))
    if key==90:
        df_90 = grouped_df.get_group(90)
        print(len(df_90))
    if key==120:
        df_120 = grouped_df.get_group(120)
        print(len(df_120))
        



89
101
92
92
95


In [26]:


dfinal = sorted_result_df.copy()
dfinal['Id']= dfinal['Id'].astype(int)
df_groupby = dfinal.groupby(['Id','segment_lenInSec']).mean()
df_groupby
#df_for_classification = df_groupby.merge(InfoLabels, on ='Id')
#df_for_classification

df_copy = df_groupby.reset_index(level=[0,1])

grouped_df = df_copy.groupby('segment_lenInSec')

for key, item in grouped_df:
    if key==0:
        df_0 = grouped_df.get_group(0)
    if key==30:
        df_30 = grouped_df.get_group(30)
    if key==60:
        df_60 = grouped_df.get_group(60)
    if key==90:
        df_90 = grouped_df.get_group(90)
    if key==120:
        df_120 = grouped_df.get_group(120)

df_labels_dict = {"LL" : 0 , "EL" : 1 }

df_0_classification = df_0.merge(InfoLabels, on ='Id')
df_0_classification['Labels'] = df_0_classification['Labels'].map(df_labels_dict)
df_0_classification.to_csv("df_0_classification2023.csv",index=False)
print(len(df_0_classification))


df_30_classification = df_30.merge(InfoLabels, on ='Id')
df_30_classification['Labels'] = df_30_classification['Labels'].map(df_labels_dict)
df_30_classification.to_csv("df_30_classification2023.csv",index=False)
print(len(df_30_classification))

df_60_classification = df_60.merge(InfoLabels, on ='Id')
df_60_classification['Labels'] = df_60_classification['Labels'].map(df_labels_dict)
df_60_classification.to_csv("df_60_classification2023.csv",index=False)
print(len(df_60_classification))

df_90_classification = df_90.merge(InfoLabels, on ='Id')
df_90_classification['Labels'] = df_90_classification['Labels'].map(df_labels_dict)
df_90_classification.to_csv("df_90_classification2023.csv",index=False)
print(len(df_90_classification))


df_120_classification = df_120.merge(InfoLabels, on ='Id')
df_120_classification['Labels'] = df_120_classification['Labels'].map(df_labels_dict)
df_120_classification.to_csv("df_120_classification2023.csv",index=False)
print(len(df_120_classification))

51
61
54
56
57


In [27]:
len(pd.read_csv("df_120_classification2023.csv"))

57

In [28]:
df_90count = pd.read_csv('df_90_classification2023.csv')
df_90count.groupby('Labels').mean()

,Id,segment_lenInSec,MeanNN,MedianNN,pnn20,sd1sd2,CvNN,HTI,CSI,CVI,MaxNN,MinNN
Labels,,,,,,,,,,,,
0,1045.696970,90.0,571.332415,569.138652,11.703802,0.359812,0.049437,5.647090,3.376040,3.674841,704.713147,501.484572
1,1075.173913,90.0,419.473671,417.091259,8.758629,0.399597,0.062138,5.005218,3.562715,3.574622,520.118603,347.182122


In [29]:
df_90count.groupby('Labels').std()

,Id,segment_lenInSec,MeanNN,MedianNN,pnn20,sd1sd2,CvNN,HTI,CSI,CVI,MaxNN,MinNN
Labels,,,,,,,,,,,,
0,29.494157,0.0,214.773662,213.743424,11.208240,0.156375,0.028874,2.388920,1.239546,0.529199,313.007851,198.367641
1,31.745654,0.0,175.864150,176.218008,11.862371,0.277030,0.034680,2.021418,1.619465,0.582844,225.884906,149.897395
